In [2]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from config import weather_api_key
import time

In [3]:
uber_csv = pd.read_csv("Resources/uber.csv")

uber_df = pd.DataFrame(uber_csv)

uber_df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [4]:
uber_df.rename(columns = {'Unnamed: 0':'ID', 'key':'N/A', 'fare_amount':'Charge', "pickup_datetime":"Pickup Time",  "pickup_longitude": "Pickup Lon", "pickup_latitude":"Pickup Lat",  
                          "dropoff_longitude": "Dropoff Lon", "dropoff_latitude": "Dropoff Lat", "passenger_count": "Passenger Count"}, inplace = True)

uber_df.head()


,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [5]:
sample_df = uber_df.sample(500)
sample_df.head()

,ID,N/A,Charge,Pickup Time,Pickup Lon,Pickup Lat,Dropoff Lon,Dropoff Lat,Passenger Count
163667,10337358,2014-01-29 19:30:00.000000111,12.0,2014-01-29 19:30:00 UTC,-73.982023,40.771637,-73.980967,40.747275,1
84692,44862084,2011-10-31 19:15:00.00000068,8.1,2011-10-31 19:15:00 UTC,-73.970960,40.764095,-73.951512,40.774023,2
198179,718918,2012-01-15 16:47:00.000000120,6.5,2012-01-15 16:47:00 UTC,-73.960682,40.777147,-73.970965,40.757040,2
3411,23856841,2014-01-01 12:38:03.0000001,10.0,2014-01-01 12:38:03 UTC,-74.009728,40.712482,-74.007032,40.748759,1
69051,2270624,2014-03-16 00:45:07.0000004,7.5,2014-03-16 00:45:07 UTC,-73.988599,40.731404,-74.007693,40.742747,1


In [6]:
url = f"https://history.openweathermap.org/data/2.5/history/city?"

In [ ]:
for row in sample_df:

    lat = row["Pickup Lat"]
    lon = row["Pickup Lon"]
    start = row["Pickup Time"]

    query = f"lat={lat}&lon={lon}&type=hour&start={start}&cnt=1&appid={weather_api_key}"

    response = requests.get(url+query).json()